Verificación de calidad de datos con Great Expectations
=======================================================

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
import great_expectations as ge

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets

iris = datasets.load_iris()
X = iris.data[:,:4]
y = iris.target

In [5]:
df = pd.DataFrame(X, columns=['longitud del sepalo',
                              'ancho del sepalo',
                              'longitud del petalo',
                              'ancho del pétalo'])
df.describe()

,longitud del sepalo,ancho del sepalo,longitud del petalo,ancho del pétalo
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [6]:
ge_df = ge.from_pandas(df)

In [ ]:
import json

with open( "my_expectation_file.json", "w") as my_file:
    my_file.write(
        json.dumps(my_df.get_expectation_suite().to_json_dict())
    )